<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mod-4-Project---Starter-Notebook" data-toc-modified-id="Mod-4-Project---Starter-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mod 4 Project - Starter Notebook</a></span></li><li><span><a href="#Some-Notes-Before-Starting" data-toc-modified-id="Some-Notes-Before-Starting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Some Notes Before Starting</a></span><ul class="toc-item"><li><span><a href="#Wide-Format-vs-Long-Format" data-toc-modified-id="Wide-Format-vs-Long-Format-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Wide Format vs Long Format</a></span></li></ul></li><li><span><a href="#Helper-Functions-Provided" data-toc-modified-id="Helper-Functions-Provided-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper Functions Provided</a></span></li><li><span><a href="#Step-1:-Load-the-Data/Filtering-for-Chosen-Zipcodes" data-toc-modified-id="Step-1:-Load-the-Data/Filtering-for-Chosen-Zipcodes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 1: Load the Data/Filtering for Chosen Zipcodes</a></span></li><li><span><a href="#Step-2:-Data-Preprocessing" data-toc-modified-id="Step-2:-Data-Preprocessing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 2: Data Preprocessing</a></span></li><li><span><a href="#Step-3:-EDA-and-Visualization" data-toc-modified-id="Step-3:-EDA-and-Visualization-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 3: EDA and Visualization</a></span></li><li><span><a href="#Step-4:-Reshape-from-Wide-to-Long-Format" data-toc-modified-id="Step-4:-Reshape-from-Wide-to-Long-Format-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 4: Reshape from Wide to Long Format</a></span></li><li><span><a href="#Step-5:-ARIMA-Modeling" data-toc-modified-id="Step-5:-ARIMA-Modeling-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Step 5: ARIMA Modeling</a></span></li><li><span><a href="#Step-6:-Interpreting-Results" data-toc-modified-id="Step-6:-Interpreting-Results-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Step 6: Interpreting Results</a></span></li></ul></div>

# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
data = pd.read_csv('zillow_data.csv')
df = pd.DataFrame(data)

In [3]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [4]:
df_LA = df[df['City'] == 'Los Angeles']

In [5]:
df_LA.sort_values(by='SizeRank', na_position='first')

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
30,96027,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,31,340600.0,341700.0,343000.0,...,1839800,1861100,1888600,1903900,1907500,1922100,1952400,1974500,1975900,1966900
75,96015,90034,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,76,231300.0,231700.0,232100.0,...,1287500,1310000,1329900,1347200,1360300,1368500,1382700,1398000,1411400,1419100
86,96025,90044,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,87,119500.0,119500.0,119400.0,...,383800,386900,390600,394300,398600,404200,409700,414300,418500,421100
107,96007,90026,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,108,138100.0,138300.0,138300.0,...,883600,895800,907700,919900,932600,947300,962600,973700,975200,972100
133,96047,90066,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,134,241100.0,241000.0,240900.0,...,1435000,1454100,1471100,1473300,1470400,1476200,1490500,1496300,1494000,1489900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7105,96426,91436,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,7106,410400.0,410400.0,410400.0,...,1540000,1558700,1576800,1579800,1577800,1587200,1601400,1616900,1639200,1654200
7578,96371,91345,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,7579,143700.0,143300.0,142900.0,...,494600,498900,503800,508800,513000,516900,520600,523600,525600,526500
7976,95995,90014,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,7977,NaN,NaN,NaN,...,640300,642700,648800,662100,681100,694700,696400,694000,691700,689600
8697,96058,90077,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,8698,762500.0,764100.0,765700.0,...,2968700,2985600,3006100,3032700,3056100,3073600,3101200,3133100,3155800,3159700


The top two in size rank are West Hollywood and Santa Monica. Makes sense. These are densley populated zip codes. 

# Step 2: Data Preprocessing

I just checked and it looks like 'RegionName' is the zipcode and 'RegionID' is extra information that we don't need. So I will go ahead and drop that column.

In [6]:
df_LA = df_LA.drop(['RegionID','SizeRank'], axis=1)

In [7]:
df_LA

,RegionName,City,State,Metro,CountyName,1996-04,1996-05,1996-06,1996-07,1996-08,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
30,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,340600.0,341700.0,343000.0,344400.0,346100.0,...,1839800,1861100,1888600,1903900,1907500,1922100,1952400,1974500,1975900,1966900
75,90034,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,231300.0,231700.0,232100.0,232400.0,232700.0,...,1287500,1310000,1329900,1347200,1360300,1368500,1382700,1398000,1411400,1419100
86,90044,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,119500.0,119500.0,119400.0,119300.0,119200.0,...,383800,386900,390600,394300,398600,404200,409700,414300,418500,421100
107,90026,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,138100.0,138300.0,138300.0,138300.0,138300.0,...,883600,895800,907700,919900,932600,947300,962600,973700,975200,972100
133,90066,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,241100.0,241000.0,240900.0,240800.0,240700.0,...,1435000,1454100,1471100,1473300,1470400,1476200,1490500,1496300,1494000,1489900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7105,91436,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,410400.0,410400.0,410400.0,410700.0,411200.0,...,1540000,1558700,1576800,1579800,1577800,1587200,1601400,1616900,1639200,1654200
7578,91345,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,143700.0,143300.0,142900.0,142500.0,142100.0,...,494600,498900,503800,508800,513000,516900,520600,523600,525600,526500
7976,90014,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,NaN,NaN,NaN,NaN,NaN,...,640300,642700,648800,662100,681100,694700,696400,694000,691700,689600
8697,90077,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,762500.0,764100.0,765700.0,767300.0,769100.0,...,2968700,2985600,3006100,3032700,3056100,3073600,3101200,3133100,3155800,3159700


In [8]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [9]:
get_datetimes(df_LA)

DatetimeIndex(['1996-06-01', '1996-07-01', '1996-08-01', '1996-09-01',
               '1996-10-01', '1996-11-01', '1996-12-01', '1997-01-01',
               '1997-02-01', '1997-03-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=263, freq=None)

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [28]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('RegionName')#.aggregate({'value':'mean'})

In [29]:
df_LA_melt = melt_data(df_LA)

In [30]:
df_LA_melt

In [18]:
cols = ['RegionName', 'City', 'State', 'Metro', 'CountyName']
df_LA_keep = df_LA[cols]

In [19]:
df_LA_keep

,RegionName,City,State,Metro,CountyName
30,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
75,90034,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
86,90044,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
107,90026,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
133,90066,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
...,...,...,...,...,...
7105,91436,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
7578,91345,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
7976,90014,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles
8697,90077,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles


In [23]:
print(np.shape(df_LA_keep))
print(np.shape(df_LA_melt))

(95, 5)
(265, 1)


# Step 5: ARIMA Modeling

# Step 6: Interpreting Results